In [0]:
# RNN을 이용한 텍스트 생성 (Text Generation using RNN)

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [0]:
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [0]:
t = Tokenizer()
t.fit_on_texts([text])
vocab_size = len(t.word_index) + 1

print(vocab_size)

12


In [0]:
t.word_index

{'가는': 8,
 '경마장에': 2,
 '고와야': 9,
 '곱다': 11,
 '그의': 6,
 '뛰고': 4,
 '말이': 1,
 '법이다': 7,
 '오는': 10,
 '있는': 3,
 '있다': 5}

In [0]:
sequences = list()
for line in text.split('\n'):
  print(line)
  encoded = t.texts_to_sequences([line])[0]
  print(encoded)
  for i in range(1, len(encoded)):
    print('i -- ', i)
    sequence = encoded[:i+1]
    print('sequence -- ', sequence)
    sequences.append(sequence)

print('학습에 사용할 샘플수: %d' % len(sequences))

경마장에 있는 말이 뛰고 있다
[2, 3, 1, 4, 5]
i --  1
sequence --  [2, 3]
i --  2
sequence --  [2, 3, 1]
i --  3
sequence --  [2, 3, 1, 4]
i --  4
sequence --  [2, 3, 1, 4, 5]

[]
그의 말이 법이다
[6, 1, 7]
i --  1
sequence --  [6, 1]
i --  2
sequence --  [6, 1, 7]

[]
가는 말이 고와야 오는 말이 곱다
[8, 1, 9, 10, 1, 11]
i --  1
sequence --  [8, 1]
i --  2
sequence --  [8, 1, 9]
i --  3
sequence --  [8, 1, 9, 10]
i --  4
sequence --  [8, 1, 9, 10, 1]
i --  5
sequence --  [8, 1, 9, 10, 1, 11]

[]
학습에 사용할 샘플수: 11


In [0]:
sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [0]:
max_len = max(len(l) for l in sequences)
max_len

6

In [0]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [0]:
sequences

array([[ 0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  2,  3,  1],
       [ 0,  0,  2,  3,  1,  4],
       [ 0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  6,  1],
       [ 0,  0,  0,  6,  1,  7],
       [ 0,  0,  0,  0,  8,  1],
       [ 0,  0,  0,  8,  1,  9],
       [ 0,  0,  8,  1,  9, 10],
       [ 0,  8,  1,  9, 10,  1],
       [ 8,  1,  9, 10,  1, 11]], dtype=int32)

In [0]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [0]:
y = to_categorical(y, num_classes=vocab_size)

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [0]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1)) # 레이블을 분리하였으므로 이제 X의 길이는 5
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=1)

Epoch 1/200
1/1 [==============================] - 0s 1ms/step - loss: 2.4757 - accuracy: 0.0000e+00
Epoch 2/200
1/1 [==============================] - 0s 1ms/step - loss: 2.4649 - accuracy: 0.0000e+00
Epoch 3/200
1/1 [==============================] - 0s 889us/step - loss: 2.4538 - accuracy: 0.3636
Epoch 4/200
1/1 [==============================] - 0s 1ms/step - loss: 2.4424 - accuracy: 0.3636
Epoch 5/200
1/1 [==============================] - 0s 1ms/step - loss: 2.4306 - accuracy: 0.4545
Epoch 6/200
1/1 [==============================] - 0s 2ms/step - loss: 2.4184 - accuracy: 0.4545
Epoch 7/200
1/1 [==============================] - 0s 999us/step - loss: 2.4056 - accuracy: 0.4545
Epoch 8/200
1/1 [==============================] - 0s 1ms/step - loss: 2.3922 - accuracy: 0.4545
Epoch 9/200
1/1 [==============================] - 0s 1ms/step - loss: 2.3782 - accuracy: 0.4545
Epoch 10/200
1/1 [==============================] - 0s 2ms/step - loss: 2.3634 - accuracy: 0.4545
Epoch 11/200
1/1 

In [0]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
        print(result)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        print(word)
        print(index)
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [0]:
print(sentence_generation(model, t, '경마장에', 1))

[3]
있는
3
경마장에 있는


In [0]:
print(sentence_generation(model, t, '가는 말이', 10)) # 2번 예측

가는 말이 고와야 오는 말이 곱다 오는 말이 곱다 오는 말이 곱다


In [0]:
print(sentence_generation(model, t, '가는', 5)) # 5번 예측

가는 말이 고와야 오는 말이 곱다
